In [30]:
# Import Statements
import pandas as pd
import numpy as np

#Turn of Notebook Package Warnings
import warnings
warnings.filterwarnings('ignore')


In [31]:
#Magic Statement
%matplotlib inline

In [20]:
# File Locations
# Change these on your machine (if needed)
datadir = ""
weather_data_raw = datadir + "2019.csv"
station_data_raw = datadir + "ghcnd-stations.txt"

In [22]:
weather = pd.read_table(weather_data_raw, sep=",", header=None)
stations = pd.read_table(station_data_raw, header=None)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  if __name__ == '__main__':


FileNotFoundError: [Errno 2] File b'2019.csv' does not exist: b'2019.csv'

In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


ImportError: cannot import name 'subplots'

In [1]:
import pandas as pd
df = pd.read_csv('data.csv')

FileNotFoundError: [Errno 2] File b'data.csv' does not exist: b'data.csv'

In [5]:
#!/usr/bin/env python
from __future__ import print_function
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row

sc = SparkContext.getOrCreate ()  # appName='weatherStats')
sqlc = SQLContext ( sc )

available_years = range ( 2000, 2019 )  # all data from 2000-2018


def mkdf(filename) :
    """
    Read filename (or glob; things like '20??.csv' will work) and return a
    handle to a PySpark DataFrame
    """
    raw = sc.textFile ( filename )
    data = raw.map ( lambda x : x.split ( ',' ) )
    table = data.map ( lambda r : Row ( sta = r[0], date = r[1], meas = r[2],
                                        degc = int ( r[3] ), m = r[4], q = r[5], s = r[6],
                                        time = r[7] ) )
    df = sqlc.createDataFrame ( table )
    return df.filter ( df.q == '' )  # prune measurements w/ quality problems


def run(years=available_years) :
    """
    Run analyses on individual years in sequence
    """
    import pyspark.sql.functions as sqlf

    # allow passing a single year or a list of them
    if not type ( years ) is list :
        years = [years]

    for year in years :
        if not year in available_years :
            raise RuntimeError ( 'Sorry, %s is not available in the dataset.' % year )

        df = mkdf ( 'data/%s.csv' % year )

        print ( "\n%s\n====\n" % year )

        # Lowest TMIN
        r = df.filter ( df.meas == 'TMIN' ).groupBy ().min ( 'degc' ).first ()
        print ( 'Min TMIN = %0.1f deg C' % (r['min(degc)'] / 10.0) )

        # Highest TMAX
        r = df.filter ( df.meas == 'TMAX' ).groupBy ().max ( 'degc' ).first ()
        print ( 'Max TMAX = %0.1f deg C' % (r['max(degc)'] / 10.0) )

        # Average TMIN
        r = df.filter ( df.meas == 'TMIN' ).groupBy ().avg ( 'degc' ).first ()
        print ( 'Avg TMIN = %0.1f deg C' % (r['avg(degc)'] / 10.0) )

        # Average TMAX
        r = df.filter ( df.meas == 'TMAX' ).groupBy ().avg ( 'degc' ).first ()
        print ( 'Avg TMAX = %0.1f deg C' % (r['avg(degc)'] / 10.0) )

        # Median TMIN
        r = df.filter ( df.meas == 'TMIN' ).approxQuantile ( 'degc', [0.5], 0.25 )
        print ( 'Median TMIN = %0.1f deg C' % (r[0] / 10.0) )

        # Median TMAX
        r = df.filter ( df.meas == 'TMAX' ).approxQuantile ( 'degc', [0.5], 0.25 )
        print ( 'Median TMAX = %0.1f deg C' % (r[0] / 10.0) )

        # Five hottest stations (on average)
        fivehot = df.filter ( df.meas == 'TMAX' ) \
            .groupBy ( df.sta ) \
            .agg ( sqlf.avg ( 'degc' ) ) \
            .sort ( sqlf.desc ( 'avg(degc)' ) ) \
            .limit ( 5 ).collect ()
        print ()
        i = 1
        for s in fivehot :
            t = float ( s['avg(degc)'] ) / 10.0
            print ( 'Hottest station #%s: %s - %0.1f deg C'
                    % (i, s.sta, t) )
            i = i + 1

        # Five coldest stations (on average)
        fivecold = df.filter ( df.meas == 'TMIN' ) \
            .groupBy ( df.sta ) \
            .agg ( sqlf.avg ( 'degc' ) ) \
            .sort ( sqlf.asc ( 'avg(degc)' ) ) \
            .limit ( 5 ).collect ()
        print ()
        i = 1
        for s in fivecold :
            t = float ( s['avg(degc)'] ) / 10.0
            print ( 'Coldest station #%s: %s - %0.1f deg C'
                    % (i, s.sta, t) )
            i = i + 1


def run_whole_dataset() :
    """
    Run analyses over the entire dataset
    """
    import pyspark.sql.functions as sqlf
    from datetime import datetime as dt

    # Hottest and coldest day and corresponding weather stations in the
    # entire dataset
    print ( "\nEntire dataset (2000-2018)\n==========================\n" )
    print ( '  * Loading all datasets into a single DataFrame...' )
    df = mkdf ( 'data/20??.csv' )

    # coldest station
    print ( '  * Computing coldest station for entire dataset...\n' )
    coldest = df.filter ( df.meas == 'TMIN' ).groupBy ( 'sta', 'date' ).min ( 'degc' ) \
        .sort ( sqlf.asc ( 'min(degc)' ) ).first ()

    date = dt.strptime ( coldest.date, '%Y%m%d' ).strftime ( '%d %b %Y' )

    print ( 'Coldest station was %s on %s: %0.1f deg C'
            % (coldest.sta, date, float ( coldest['min(degc)'] ) / 10.0) )

    # and now the hottest
    print ( '\n  * Computing hottest station for entire dataset...\n' )
    hottest = df.filter ( df.meas == 'TMAX' ).groupBy ( 'sta', 'date' ).max ( 'degc' ) \
        .sort ( sqlf.desc ( 'max(degc)' ) ).first ()

    date = dt.strptime ( hottest.date, '%Y%m%d' ).strftime ( '%d %b %Y' )

    print ( 'Hottest station was %s on %s: %0.1f deg C'
            % (hottest.sta, date, float ( hottest['max(degc)'] ) / 10.0) )

    # Median TMIN
    print ( '\n  * Computing median TMIN for entire dataset...\n' )
    TMIN_m = df.filter ( df.meas == 'TMIN' ).approxQuantile ( 'degc', [0.5], 0.25 )
    print ( 'Median TMIN for the entire dataset: %0.1f deg C' % (TMIN_m[0] / 10.0) )

    # Median TMAX
    print ( '\n  * Computing median TMAX for entire dataset...\n' )
    TMAX_m = df.filter ( df.meas == 'TMAX' ).approxQuantile ( 'degc', [0.5], 0.25 )
    print ( 'Median TMAX for the entire dataset: %0.1f deg C' % (TMAX_m[0] / 10.0) )


if __name__ == '__main__' :
    run ()


ModuleNotFoundError: No module named 'pyspark'

In [1]:
name = 'Lizz'

In [2]:
print(name[0:2])

Li


In [3]:
var = '01234567'

In [4]:
print(var[::2])

0246


In [10]:
import pandas as pd
url = "http://www.fa-technik.adfc.de/code/opengeodb/DE.tab"

df = pd.read_csv(url, sep='\t', engine='python')

df.head(5)

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [37]:
path = "C:\temp\data.csv"
df.to_csv(path)


In [13]:
df.describe(include = "all")

,#loc_id,ags,ascii,name,lat,lon,amt,plz,vorwahl,einwohner,flaeche,kz,typ,level,of,invalid
count,61135.000000,61052,61121,61121,60981.000000,60981.000000,10012,57907,13024,1.284300e+04,13020.000000,60686,15289,61133.000000,61125.000000,880.000000
unique,NaN,11427,44983,53607,NaN,NaN,1592,8520,4543,NaN,NaN,425,96,NaN,NaN,NaN
top,NaN,02000000,BERG,Berg,NaN,NaN,Samtgemeinde Clenze,84494,030,NaN,NaN,RO,Gemeinde,NaN,NaN,NaN
freq,NaN,114,71,47,NaN,NaN,88,83,112,NaN,NaN,926,9001,NaN,NaN,NaN
mean,95652.790169,NaN,NaN,NaN,50.400102,10.341184,NaN,NaN,NaN,2.179012e+04,111.903779,NaN,NaN,7.514861,17648.927182,0.181818
std,46585.742810,NaN,NaN,NaN,1.951402,1.960405,NaN,NaN,NaN,7.682999e+05,3293.166256,NaN,NaN,0.843488,16317.453889,0.385914
min,1.000000,NaN,NaN,NaN,34.000000,5.866667,NaN,NaN,NaN,1.000000e+00,0.000000,NaN,NaN,2.000000,1.000000,0.000000
25%,81156.500000,NaN,NaN,NaN,48.750000,8.850000,NaN,NaN,NaN,5.855000e+02,8.000000,NaN,NaN,7.000000,13902.000000,0.000000
50%,115144.000000,NaN,NaN,NaN,50.033300,10.383333,NaN,NaN,NaN,1.533000e+03,18.000000,NaN,NaN,8.000000,18376.000000,0.000000
75%,130427.500000,NaN,NaN,NaN,51.921561,11.950000,NaN,NaN,NaN,4.847000e+03,39.000000,NaN,NaN,8.000000,22852.000000,0.000000


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61135 entries, 0 to 61134
Data columns (total 16 columns):
#loc_id      61135 non-null int64
ags          61052 non-null object
ascii        61121 non-null object
name         61121 non-null object
lat          60981 non-null float64
lon          60981 non-null float64
amt          10012 non-null object
plz          57907 non-null object
vorwahl      13024 non-null object
einwohner    12843 non-null float64
flaeche      13020 non-null float64
kz           60686 non-null object
typ          15289 non-null object
level        61133 non-null float64
of           61125 non-null float64
invalid      880 non-null float64
dtypes: float64(7), int64(1), object(8)
memory usage: 7.5+ MB


In [17]:
df.head(5)

,#loc_id,ags,ascii,name,lat,lon,amt,plz,vorwahl,einwohner,flaeche,kz,typ,level,of,invalid
0,105,D,DEUTSCHLAND,Bundesrepublik Deutschland,51.16766,10.42498,NaN,NaN,NaN,82175684.0,357104.0,D,NaN,2.0,104.0,NaN
1,80076,16076087,KLEINWOLSCHENDORF,Kleinwolschendorf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,26599.0,0.0
2,80085,16076087,PAHREN,Pahren,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ortsteil,8.0,26599.0,0.0
3,80070,16076087,DOERTENDORF,Dörtendorf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ortsteil,8.0,25089.0,0.0
4,80082,16076087,MEHLA,Mehla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ortsteil,8.0,25089.0,0.0


In [38]:
df.dropna(subset=["kz", "amt", "invalid"], axis=0)

,#loc_id,ags,ascii,name,lat,lon,amt,plz,vorwahl,einwohner,flaeche,kz,typ,level,of,invalid
13614,16367,15083215,ETINGEN,Etingen,52.393160,11.170410,Verwaltungsgemeinschaft Oebisfelde-Calvörde,39359,039002,533.0,16.93,BK,Gemeinde,6.0,541.0,1.0
14473,50061,16067085,GUENTHERSLEBEN-WECHMAR,Günthersleben-Wechmar,50.900000,10.775000,Verwaltungsgemeinschaft Günthersleben-Wechmar,99869,036256,3141.0,27.00,GTH,Gemeinde,6.0,342.0,1.0
60408,150732,13071027,LEHNENHOF,Lehnenhof,53.925672,12.846923,Amt Dargun,17159,039959,NaN,NaN,DM,Ortsteil,8.0,15441.0,0.0
60411,150739,13071142,KLOCKOW-DORF,Klockow-Dorf,53.701178,12.951937,Amt Stavenhagen,17153,039954,NaN,NaN,DM,Ortsteil,7.0,23041.0,0.0
60412,150741,13071142,KOELPIN,Kölpin,53.710933,12.858853,Amt Stavenhagen,17153,039954,NaN,NaN,DM,Ortsteil,7.0,23041.0,0.0
60413,150744,13071142,NEUBAUHOF,Neubauhof,53.683759,12.883981,Amt Stavenhagen,17153,039954,NaN,NaN,DM,Dorf,7.0,23041.0,0.0
60414,150746,13071142,WUESTGRABOW,Wüstgrabow,53.726703,12.880526,Amt Stavenhagen,17153,039954,NaN,NaN,DM,Dorf,7.0,23041.0,0.0
60420,150764,12061061,SIEDLUNG,Siedlung,51.890300,14.149800,Amt Lieberose/Oberspreewald,15913,035475,NaN,NaN,LDS,Ortsteil,9.0,15254.0,0.0
60421,150766,12061061,MUEHLENDORF,Mühlendorf,51.884700,14.145500,Amt Lieberose / Oberspreewald,15913,035475,NaN,NaN,LDS,Ortsteil,9.0,15254.0,0.0
60422,150768,12061061,NEU-BYHLEGUHRE,Neu-Byhleguhre,51.870900,14.191800,Amt Lieberose / Oberspreewald,15913,035475,NaN,NaN,LDS,Ortsteil,9.0,15254.0,0.0


In [34]:
df.to_csv("cities1.csv", index=False)

ValueError: Invalid file path or buffer object type: <class 'pandas.core.frame.DataFrame'>

In [46]:
s = df['typ']

In [47]:
s

0                             NaN
1                             NaN
2                        Ortsteil
3                        Ortsteil
4                        Ortsteil
5                        Ortsteil
6                             NaN
7                        Ortsteil
8                        Ortsteil
9                        Ortsteil
10                       Ortsteil
11                            NaN
12                            NaN
13                            NaN
14                            NaN
15                            NaN
16                            NaN
17                            NaN
18                            NaN
19                            NaN
20                            NaN
21                            NaN
22                            NaN
23                            NaN
24                            NaN
25                            NaN
26                       Ortsteil
27                       Ortsteil
28                       Ortsteil
29            

In [57]:
df.loc[df['typ'] == 'Stadt']

,#loc_id,ags,ascii,name,lat,lon,amt,plz,vorwahl,einwohner,flaeche,kz,typ,level,of,invalid
37,466,01001,FLENSBURG,Flensburg,54.780470,9.435220,NaN,NaN,NaN,NaN,56.00,FL,Stadt,5.0,119.0,NaN
38,16534,01001000,FLENSBURG,Flensburg,54.780470,9.435220,NaN,"24937,24939,24941,24943,24944",0461,86006.0,56.00,FL,Stadt,6.0,466.0,NaN
43,474,01004,NEUMUENSTER,Neumünster,54.074040,9.984530,NaN,NaN,NaN,NaN,72.00,NMS,Stadt,5.0,119.0,NaN
44,21517,01004000,NEUMUENSTER,"Neumünster, Holstein",54.074020,9.984512,NaN,"24534,24536,24537,24539",04321,78128.0,72.00,NMS,Stadt,6.0,474.0,NaN
54,15051,01051011,BRUNSBUETTEL,Brunsbüttel,53.897863,9.128617,NaN,25541,04852,13789.0,65.00,HEI,Stadt,6.0,258.0,NaN
81,18047,01051044,HEIDE,Heide / Holstein,54.195730,9.091090,NaN,25746,0481,20716.0,32.00,HEI,Stadt,6.0,258.0,NaN
104,20675,01051072,MARNE,"Marne, Holstein",53.951127,9.008806,NaN,25709,04851,6018.0,5.00,HEI,Stadt,6.0,258.0,NaN
106,20799,01051074,MELDORF,Meldorf (Holstein),54.091950,9.069270,NaN,25704,04832,7655.0,21.00,HEI,Stadt,6.0,258.0,NaN
149,26008,01051127,WESSELBUREN,Wesselburen,54.211900,8.920840,NaN,25764,04833,3112.0,5.00,HEI,Stadt,6.0,258.0,NaN
195,16875,01053032,GEESTHACHT,Geesthacht,53.430180,10.374100,NaN,21502,04152,29403.0,33.00,RZ,Stadt,6.0,463.0,NaN


In [81]:
import pandas as pd
url = "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt"

dfstations = pd.read_csv(url, sep='\t', engine='python')


dfstations.head(5)

# bydefault splitting is done on the basis of single space. 
print("\nSplitting 'Name' column into two different columns :\n", 
                                  dfstations.Name.str.split(expand=True)) 

AttributeError: 'DataFrame' object has no attribute 'Name'

In [7]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_7430ea7319b540a9a8f687c4c52b3576 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='LnJC4G_Dt-onn470r5O2hV4xtYu7ueglzlSe5DqymQqz',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_7430ea7319b540a9a8f687c4c52b3576.get_object(Bucket='myproject-donotdelete-pr-dnoa0xuakl436y',Key='stations_data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()
# Erstellung einer Header-Liste
headers = ["ID","LATITUDE","LONGITUDE","ELEVATION","STATE", "NAME","GSN FLAG",
         "HCN/CRN FLAG","WMO ID"]
print("headers\n", headers)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 115, saw 2


In [70]:

dfstations.head(10)

,ACW00011604 17.1167 -61.7833 10.1 ST JOHNS COOLIDGE FLD
0,ACW00011647 17.1333 -61.7833 19.2 ST JO...
1,AE000041196 25.3330 55.5170 34.0 SHARJ...
2,AEM00041194 25.2550 55.3640 10.4 DUBAI...
3,AEM00041217 24.4330 54.6510 26.8 ABU D...
4,AEM00041218 24.2620 55.6090 264.9 AL AI...
5,AF000040930 35.3170 69.0170 3366.0 NORTH...
6,AFM00040938 34.2100 62.2280 977.2 HERAT...
7,AFM00040948 34.5660 69.2120 1791.3 KABUL...
8,AFM00040990 31.5000 65.8500 1010.0 KANDA...
9,AG000060390 36.7167 3.2500 24.0 ALGER...


In [86]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_2 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-1a3e9502-0f8b-475f-86bc-2106d0e4ca15',
    'IBM_API_KEY_ID': 'LnJC4G_Dt-onn470r5O2hV4xtYu7ueglzlSe5DqymQqz',
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET': 'myproject-donotdelete-pr-dnoa0xuakl436y',
    'FILE': 'stations_all.csv'
}


ParserError: Error tokenizing data. C error: Expected 1 fields in line 115, saw 2


In [2]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-1a3e9502-0f8b-475f-86bc-2106d0e4ca15',
    'IBM_API_KEY_ID': 'LnJC4G_Dt-onn470r5O2hV4xtYu7ueglzlSe5DqymQqz',
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET': 'myproject-donotdelete-pr-dnoa0xuakl436y',
    'FILE': 'stations_all.csv'
}
body = client_7430ea7319b540a9a8f687c4c52b3576.get_object(Bucket='myproject-donotdelete-pr-dnoa0xuakl436y',Key='2019.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head(5)


NameError: name 'client_7430ea7319b540a9a8f687c4c52b3576' is not defined

In [5]:
credentials_3 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-1a3e9502-0f8b-475f-86bc-2106d0e4ca15',
    'IBM_API_KEY_ID': 'LnJC4G_Dt-onn470r5O2hV4xtYu7ueglzlSe5DqymQqz',
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET': 'myproject-donotdelete-pr-dnoa0xuakl436y',
    'FILE': '2019.csv'
body = client_7430ea7319b540a9a8f687c4c52b3576.get_object(Bucket='myproject-donotdelete-pr-dnoa0xuakl436y',Key='2019.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head()


NameError: name 'client_7430ea7319b540a9a8f687c4c52b3576' is not defined

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
client_7430ea7319b540a9a8f687c4c52b3576 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='LnJC4G_Dt-onn470r5O2hV4xtYu7ueglzlSe5DqymQqz',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_7430ea7319b540a9a8f687c4c52b3576.get_object(Bucket='myproject-donotdelete-pr-dnoa0xuakl436y',Key='2019.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
df_data_3.head(2)